# Preparation

In [1]:
# Pip install method (recommended)
!pip install ultralytics -qq
!pip install gdown -qq

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.61 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6095.9/8062.4 GB disk)


## Download Data nama datanya di GDrive Jethro Data_copied.zip

In [2]:
# Deprecated
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=19scGP0Slw4Qc57iSj3_ZcBftw29vb18T' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=19scGP0Slw4Qc57iSj3_ZcBftw29vb18T" -O data.zip && rm -rf /tmp/cookies.txt

In [3]:
!gdown --fuzzy https://drive.google.com/file/d/19scGP0Slw4Qc57iSj3_ZcBftw29vb18T/view?usp=drive_link -O data.zip

Downloading...
From (original): https://drive.google.com/uc?id=19scGP0Slw4Qc57iSj3_ZcBftw29vb18T
From (redirected): https://drive.google.com/uc?id=19scGP0Slw4Qc57iSj3_ZcBftw29vb18T&confirm=t&uuid=30f1ca27-d79c-437c-bcee-b6467b23d5d4
To: /kaggle/working/data.zip
100%|████████████████████████████████████████| 471M/471M [00:05<00:00, 92.4MB/s]


In [4]:
!unzip -q -o ./data.zip -d .

In [5]:
import os

# Directory to save the images
output_directory = './content/output'
# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

In [6]:
source_dataset={
    'ANTARANEWS':'/kaggle/working/content/data_copied/ANTARANEWS',
    'BISNIS':'/kaggle/working/content/data_copied/BISNIS',
    'SUARA':'/kaggle/working/content/data_copied/SUARA',
    'DETIK':'/kaggle/working/content/data_copied/DETIK',
    'CNN':'/kaggle/working/content/data_copied/CNN',
}

In [7]:
import os

# Folder untuk source_dataset
source_folders = [
    '/kaggle/working/content/data_copied/ANTARANEWS',
    '/kaggle/working/content/data_copied/BISNIS',
    '/kaggle/working/content/data_copied/SUARA',
    '/kaggle/working/content/data_copied/DETIK',
    '/kaggle/working/content/data_copied/CNN'
]

# Membuat folder
for folder in source_folders:
    os.makedirs(folder, exist_ok=True)

print("Folder berhasil dibuat!")


Folder berhasil dibuat!


## Download Model Deteksi Wajah Capres & Wapres dan Logo Parpol

In [8]:
# Deprecated
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1_SpA4EyLs8qQunEX5aCcLO30BmxB_qXR' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1_SpA4EyLs8qQunEX5aCcLO30BmxB_qXR" -O model_capres.pt && rm -rf /tmp/cookies.txt

In [9]:
# Deprecated
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1N8jCw9yuiek8nehiNjSaTXdg_0GJ1-EJ' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1N8jCw9yuiek8nehiNjSaTXdg_0GJ1-EJ" -O model_parpol.pt && rm -rf /tmp/cookies.txt

In [10]:
# Model Deteksi Wajah Capres & Wapres
!gdown --fuzzy https://drive.google.com/file/d/1_SpA4EyLs8qQunEX5aCcLO30BmxB_qXR/view?usp=drive_link -O model_capres.pt

Downloading...
From (original): https://drive.google.com/uc?id=1_SpA4EyLs8qQunEX5aCcLO30BmxB_qXR
From (redirected): https://drive.google.com/uc?id=1_SpA4EyLs8qQunEX5aCcLO30BmxB_qXR&confirm=t&uuid=9ba434ef-3452-4bb9-8fea-8d882748c592
To: /kaggle/working/model_capres.pt
100%|█████████████████████████████████████████| 137M/137M [00:00<00:00, 155MB/s]


In [11]:
# Model Deteksi Logo Parpol
!gdown --fuzzy https://drive.google.com/file/d/1N8jCw9yuiek8nehiNjSaTXdg_0GJ1-EJ/view?usp=drive_link -O model_parpol.pt

Downloading...
From (original): https://drive.google.com/uc?id=1N8jCw9yuiek8nehiNjSaTXdg_0GJ1-EJ
From (redirected): https://drive.google.com/uc?id=1N8jCw9yuiek8nehiNjSaTXdg_0GJ1-EJ&confirm=t&uuid=cbabb965-12ad-4f30-8a89-593410507e17
To: /kaggle/working/model_parpol.pt
100%|██████████████████████████████████████| 87.7M/87.7M [00:00<00:00, 94.1MB/s]


# Information Retrieval

## Capres & Cawapres Detection

In [12]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model_capres = YOLO('/kaggle/working/model_capres.pt').cuda()

In [13]:
import torch

print(next(model_capres.parameters()).device)

cuda:0


In [14]:
import pandas as pd
import os
from tqdm import tqdm

# Create a list to store the data
data = []
names = model_capres.names

# Iterate through source_dataset
for source_name, source_path in tqdm(source_dataset.items(), desc="Processing Images"):
    # Perform prediction and save results
    results_capres = model_capres.predict(source_path, save=True, conf=0.5)

    # Iterate through the results
    for i, result in enumerate(results_capres):
        # Get the original image path
        original_path = os.path.join(source_path, os.listdir(source_path)[i])

        # Get the predicted classes
        predicted_classes = [names[int(c)] for c in result.boxes.cls]

        # Append data to the list
        data.append({
            'source_name': source_name,
            'originalPath': original_path,
            'CapresDetect': predicted_classes,
        })

# Create a DataFrame from the list of data
df_capres = pd.DataFrame(data)

Processing Images:   0%|          | 0/5 [00:00<?, ?it/s]



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1159 /kaggle/working/content/data_copied/ANTARANEWS/03378100-AB24-4F27-8ECF-10FFBA224608.jpeg: 448x640 1 cakimin, 1 prabowo, 71.6ms
image 2/1159 /kaggle/working/content/data_copied/ANTARANEWS/089AA8A2-6541-45B8-96D7-7F29E521EFD6.jpeg: 448x640 (no detections), 30.2ms
image 3/1159 /kaggle/working/content/data_copied/ANTARANEWS/09cafa4c-b8d4-4874-9f2f-a5bef51c7dea.jpeg: 448x640 1 prabowo, 30.1ms
image 4/1159 /kaggle/working/content/data_copied/ANT

Processing Images:  20%|██        | 1/5 [01:15<05:02, 75.72s/it]


image 1/526 /kaggle/working/content/data_copied/BISNIS/image_1.jpg: 448x640 (no detections), 27.5ms
image 2/526 /kaggle/working/content/data_copied/BISNIS/image_10.jpg: 448x640 (no detections), 27.4ms
image 3/526 /kaggle/working/content/data_copied/BISNIS/image_100.jpg: 256x640 1 anies, 2 cakimins, 1 mahfud, 94.2ms
image 4/526 /kaggle/working/content/data_copied/BISNIS/image_101.jpg: 448x640 1 prabowo, 28.2ms
image 5/526 /kaggle/working/content/data_copied/BISNIS/image_102.jpg: 448x640 1 prabowo, 27.4ms
image 6/526 /kaggle/working/content/data_copied/BISNIS/image_103.jpg: 416x640 1 mahfud, 55.5ms
image 7/526 /kaggle/working/content/data_copied/BISNIS/image_104.jpg: 448x640 1 cakimin, 1 ganjar, 1 prabowo, 28.1ms
image 8/526 /kaggle/working/content/data_copied/BISNIS/image_105.jpg: 448x640 1 gibran, 27.5ms
image 9/526 /kaggle/working/content/data_copied/BISNIS/image_106.jpg: 448x640 (no detections), 27.4ms
image 10/526 /kaggle/working/content/data_copied/BISNIS/image_107.jpg: 448x640 (n

Processing Images:  40%|████      | 2/5 [01:44<02:24, 48.32s/it]

...

Processing Images:  60%|██████    | 3/5 [02:18<01:23, 41.82s/it]



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1053 /kaggle/working/content/data_copied/DETIK/image_1.jpg: 480x640 1 mahfud, 60.0ms
image 2/1053 /kaggle/working/content/data_copied/DETIK/image_10.jpg: 512x640 (no detections), 56.9ms
image 3/1053 /kaggle/working/content/data_copied/DETIK/image_100.jpg: 480x640 2 mahfuds, 28.9ms
image 4/1053 /kaggle/working/content/data_copied/DETIK/image_1000.jpg: 480x640 1 ganjar, 28.1ms
image 5/1053 /kaggle/working/content/data_copied/DETIK/image_1001.jpg:

Processing Images:  80%|████████  | 4/5 [03:19<00:49, 49.33s/it]


image 1/800 /kaggle/working/content/data_copied/CNN/image1.jpeg: 384x640 1 cakimin, 25.3ms
image 2/800 /kaggle/working/content/data_copied/CNN/image10.jpeg: 384x640 (no detections), 24.2ms
image 3/800 /kaggle/working/content/data_copied/CNN/image100.jpg: 384x640 (no detections), 24.2ms
image 4/800 /kaggle/working/content/data_copied/CNN/image101.jpeg: 384x640 1 prabowo, 24.2ms
image 5/800 /kaggle/working/content/data_copied/CNN/image102.jpeg: 384x640 (no detections), 24.2ms
image 6/800 /kaggle/working/content/data_copied/CNN/image103.jpeg: 384x640 (no detections), 24.1ms
image 7/800 /kaggle/working/content/data_copied/CNN/image104.jpeg: 384x640 (no detections), 24.3ms
image 8/800 /kaggle/working/content/data_copied/CNN/image105.jpeg: 384x640 (no detections), 24.3ms
image 9/800 /kaggle/working/content/data_copied/CNN/image106.jpeg: 384x640 1 mahfud, 24.3ms
image 10/800 /kaggle/working/content/data_copied/CNN/image107.png: 384x640 (no detections), 24.3ms
image 11/800 /kaggle/working/con

Processing Images: 100%|██████████| 5/5 [03:49<00:00, 45.91s/it]


In [15]:
df_capres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4398 entries, 0 to 4397
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   source_name   4398 non-null   object
 1   originalPath  4398 non-null   object
 2   CapresDetect  4398 non-null   object
dtypes: object(3)
memory usage: 103.2+ KB


In [16]:
# !zip images.zip image_617.jpg image_807.jpg image_998.jpg image_238.jpg image_428.jpg image_618.jpg image_808.jpg image_999.jpg image_239.jpg image_429.jpg image_619.jpg image_809.jpg image_24.jpg image_43.jpg image_62.jpg image_81.jpg

In [17]:
!zip -r /kaggle/working/capres_wapres_pred.zip /kaggle/working/runs/detect/predict

  adding: kaggle/working/runs/detect/predict/ (stored 0%)
  adding: kaggle/working/runs/detect/predict/IMG_20231202_173210.jpg (deflated 6%)
  adding: kaggle/working/runs/detect/predict/image790.jpg (deflated 3%)
  adding: kaggle/working/runs/detect/predict/image607.jpg (deflated 2%)
  adding: kaggle/working/runs/detect/predict/image_600.jpg (deflated 4%)
  adding: kaggle/working/runs/detect/predict/Jelang-Uji-Coba-PLTS-Terapung-Cirata-260923-rai-11.jpg (deflated 6%)
  adding: kaggle/working/runs/detect/predict/image_622.jpg (deflated 3%)
  adding: kaggle/working/runs/detect/predict/image_849.jpg (deflated 11%)
  adding: kaggle/working/runs/detect/predict/IMG-20231205-WA0028_1.jpg (deflated 4%)
  adding: kaggle/working/runs/detect/predict/image_319.jpg (deflated 4%)
  adding: kaggle/working/runs/detect/predict/image534.jpg (deflated 2%)
  adding: kaggle/working/runs/detect/predict/Gibran-Hadiri-Deklarasi-Dukungan-Perdana-5.jpg (deflated 4%)
  adding: kaggle/working/runs/detect/predict/

## Partai Politik Detection

In [18]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model_parpol = YOLO('/kaggle/working/model_parpol.pt').cuda()

In [19]:
import pandas as pd
import os
from tqdm import tqdm

# Create a list to store the data
data_parpol = []
names = model_parpol.names

# Iterate through source_dataset
for source_name, source_path in tqdm(source_dataset.items(), desc="Processing Images"):
    # Perform prediction
    results_parpol = model_parpol.predict(source_path, save=True, conf=0.2)

    # Iterate through the results
    for i, result in enumerate(results_parpol):
        # Get the original path
        original_path = os.path.join(source_path, os.listdir(source_path)[i])

        # Get the predicted classes
        predicted_classes = [names[int(c)] for c in result.boxes.cls]

        # Append data to the list
        data_parpol.append({
            'source_name': source_name, 
            'originalPath': original_path, 
            'ParpolDetect': predicted_classes
        })

# Create a DataFrame from the list of data
df_parpol = pd.DataFrame(data_parpol)

Processing Images:   0%|          | 0/5 [00:00<?, ?it/s]



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1159 /kaggle/working/content/data_copied/ANTARANEWS/03378100-AB24-4F27-8ECF-10FFBA224608.jpeg: 448x640 (no detections), 57.0ms
image 2/1159 /kaggle/working/content/data_copied/ANTARANEWS/089AA8A2-6541-45B8-96D7-7F29E521EFD6.jpeg: 448x640 (no detections), 19.4ms
image 3/1159 /kaggle/working/content/data_copied/ANTARANEWS/09cafa4c-b8d4-4874-9f2f-a5bef51c7dea.jpeg: 448x640 (no detections), 19.4ms
image 4/1159 /kaggle/working/content/data_copied/AN

Processing Images:  20%|██        | 1/5 [01:01<04:06, 61.68s/it]


image 1/526 /kaggle/working/content/data_copied/BISNIS/image_1.jpg: 448x640 2 PDIPs, 17.8ms
image 2/526 /kaggle/working/content/data_copied/BISNIS/image_10.jpg: 448x640 (no detections), 17.7ms
image 3/526 /kaggle/working/content/data_copied/BISNIS/image_100.jpg: 256x640 (no detections), 55.9ms
image 4/526 /kaggle/working/content/data_copied/BISNIS/image_101.jpg: 448x640 (no detections), 18.2ms
image 5/526 /kaggle/working/content/data_copied/BISNIS/image_102.jpg: 448x640 (no detections), 17.6ms
image 6/526 /kaggle/working/content/data_copied/BISNIS/image_103.jpg: 416x640 (no detections), 55.5ms
image 7/526 /kaggle/working/content/data_copied/BISNIS/image_104.jpg: 448x640 (no detections), 18.2ms
image 8/526 /kaggle/working/content/data_copied/BISNIS/image_105.jpg: 448x640 1 PSI, 17.6ms
image 9/526 /kaggle/working/content/data_copied/BISNIS/image_106.jpg: 448x640 (no detections), 17.6ms
image 10/526 /kaggle/working/content/data_copied/BISNIS/image_107.jpg: 448x640 (no detections), 17.6ms

Processing Images:  40%|████      | 2/5 [01:25<01:57, 39.24s/it]

...

Processing Images:  60%|██████    | 3/5 [01:53<01:08, 34.26s/it]



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1053 /kaggle/working/content/data_copied/DETIK/image_1.jpg: 480x640 (no detections), 60.9ms
image 2/1053 /kaggle/working/content/data_copied/DETIK/image_10.jpg: 512x640 (no detections), 56.1ms
image 3/1053 /kaggle/working/content/data_copied/DETIK/image_100.jpg: 480x640 (no detections), 19.0ms
image 4/1053 /kaggle/working/content/data_copied/DETIK/image_1000.jpg: 480x640 (no detections), 18.2ms
image 5/1053 /kaggle/working/content/data_copied/D

Processing Images:  80%|████████  | 4/5 [02:43<00:40, 40.40s/it]


image 1/800 /kaggle/working/content/data_copied/CNN/image1.jpeg: 384x640 (no detections), 19.4ms
image 2/800 /kaggle/working/content/data_copied/CNN/image10.jpeg: 384x640 (no detections), 18.2ms
image 3/800 /kaggle/working/content/data_copied/CNN/image100.jpg: 384x640 (no detections), 18.2ms
image 4/800 /kaggle/working/content/data_copied/CNN/image101.jpeg: 384x640 (no detections), 18.2ms
image 5/800 /kaggle/working/content/data_copied/CNN/image102.jpeg: 384x640 1 PDIP, 18.2ms
image 6/800 /kaggle/working/content/data_copied/CNN/image103.jpeg: 384x640 (no detections), 18.2ms
image 7/800 /kaggle/working/content/data_copied/CNN/image104.jpeg: 384x640 (no detections), 18.3ms
image 8/800 /kaggle/working/content/data_copied/CNN/image105.jpeg: 384x640 (no detections), 18.2ms
image 9/800 /kaggle/working/content/data_copied/CNN/image106.jpeg: 384x640 (no detections), 18.3ms
image 10/800 /kaggle/working/content/data_copied/CNN/image107.png: 384x640 (no detections), 18.2ms
image 11/800 /kaggle/w

Processing Images: 100%|██████████| 5/5 [03:07<00:00, 37.56s/it]


In [20]:
df_parpol.head()

,source_name,originalPath,ParpolDetect
0,ANTARANEWS,/kaggle/working/content/data_copied/ANTARANEWS...,[]
1,ANTARANEWS,/kaggle/working/content/data_copied/ANTARANEWS...,[]
2,ANTARANEWS,/kaggle/working/content/data_copied/ANTARANEWS...,[]
3,ANTARANEWS,/kaggle/working/content/data_copied/ANTARANEWS...,[]
4,ANTARANEWS,/kaggle/working/content/data_copied/ANTARANEWS...,[]


In [21]:
df_parpol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4398 entries, 0 to 4397
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   source_name   4398 non-null   object
 1   originalPath  4398 non-null   object
 2   ParpolDetect  4398 non-null   object
dtypes: object(3)
memory usage: 103.2+ KB


In [22]:
!zip -r /kaggle/working/parpol_pred.zip /kaggle/working/runs/detect/predict2

  adding: kaggle/working/runs/detect/predict2/ (stored 0%)
  adding: kaggle/working/runs/detect/predict2/IMG_20231202_173210.jpg (deflated 6%)
  adding: kaggle/working/runs/detect/predict2/image790.jpg (deflated 3%)
  adding: kaggle/working/runs/detect/predict2/image607.jpg (deflated 2%)
  adding: kaggle/working/runs/detect/predict2/image_600.jpg (deflated 4%)
  adding: kaggle/working/runs/detect/predict2/Jelang-Uji-Coba-PLTS-Terapung-Cirata-260923-rai-11.jpg (deflated 6%)
  adding: kaggle/working/runs/detect/predict2/image_622.jpg (deflated 3%)
  adding: kaggle/working/runs/detect/predict2/image_849.jpg (deflated 11%)
  adding: kaggle/working/runs/detect/predict2/IMG-20231205-WA0028_1.jpg (deflated 4%)
  adding: kaggle/working/runs/detect/predict2/image_319.jpg (deflated 4%)
  adding: kaggle/working/runs/detect/predict2/image534.jpg (deflated 2%)
  adding: kaggle/working/runs/detect/predict2/Gibran-Hadiri-Deklarasi-Dukungan-Perdana-5.jpg (deflated 4%)
  adding: kaggle/working/runs/det

## Color Extraction

In [23]:
import cv2
import os

In [24]:
def extract_rgb(image_path):
    # Baca gambar
    image = cv2.imread(image_path)

    # Ubah ke format warna RGB (OpenCV membaca dalam format BGR)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Ekstrak nilai RGB
    red, green, blue = image_rgb.mean(axis=(0, 1))

    return red, green, blue

In [25]:
def create_dataframe(source_dataset):
    # Inisialisasi list untuk menyimpan data
    data = {'originalPath': [], 'Red': [], 'Green': [], 'Blue': []}

    # Loop melalui setiap path dalam source_dataset
    for source_name, image_directory in source_dataset.items():
        # List semua file gambar dalam direktori
        image_files = [f for f in os.listdir(image_directory) if f.endswith(('.png', '.jpg', '.jpeg'))]

        # Loop melalui setiap gambar dan ekstrak data RGB
        for image_file in image_files:
            image_path = os.path.join(image_directory, image_file)
            red, green, blue = extract_rgb(image_path)

            # Tambahkan data ke list
            data['originalPath'].append(image_path)
            data['Red'].append(red)
            data['Green'].append(green)
            data['Blue'].append(blue)
    # Buat DataFrame dari data
    df = pd.DataFrame(data)

    return df

In [26]:
# Buat DataFrame
df_color = create_dataframe(source_dataset)

In [27]:
df_color.head()

,originalPath,Red,Green,Blue
0,/kaggle/working/content/data_copied/ANTARANEWS...,112.030239,112.387723,114.380279
1,/kaggle/working/content/data_copied/ANTARANEWS...,86.165496,96.153711,105.770896
2,/kaggle/working/content/data_copied/ANTARANEWS...,114.839089,112.572819,109.307942
3,/kaggle/working/content/data_copied/ANTARANEWS...,163.620768,130.299019,112.290558
4,/kaggle/working/content/data_copied/ANTARANEWS...,121.798487,119.056176,122.453828


## Merge Dataframe

In [28]:
# Merge DataFrame
df_merged = pd.merge(df_capres, df_parpol, on='originalPath', how='outer')
df_ = pd.merge(df_merged, df_color, on='originalPath', how='outer')

In [29]:
df = df_.drop('source_name_x', axis=1)

In [30]:
df.head()

,originalPath,CapresDetect,source_name_y,ParpolDetect,Red,Green,Blue
0,/kaggle/working/content/data_copied/ANTARANEWS...,"[prabowo, cakimin]",ANTARANEWS,[],112.030239,112.387723,114.380279
1,/kaggle/working/content/data_copied/ANTARANEWS...,[],ANTARANEWS,[],86.165496,96.153711,105.770896
2,/kaggle/working/content/data_copied/ANTARANEWS...,[prabowo],ANTARANEWS,[],114.839089,112.572819,109.307942
3,/kaggle/working/content/data_copied/ANTARANEWS...,[mahfud],ANTARANEWS,[],163.620768,130.299019,112.290558
4,/kaggle/working/content/data_copied/ANTARANEWS...,[],ANTARANEWS,[],121.798487,119.056176,122.453828


In [31]:
# Menyimpan DataFrame ke dalam file CSV
df.to_csv('dataset.csv', index=False)